<center><strong style="font-size:40px">Binary Classifier</strong></center>

In this notebook, we show how we train the binary classification learners using `mlr3` package. We will mainly train a penalised logistic regression (from `glmnet`package) and a gradient boosted tree (using `xgboost`). In a later stage, we could try to use `catboost` without converting the categorical features

# Data
The data has been precomputed using the `get_training_data()` of the `Spadl` class and was stored in RDS format. Due to its relative big size, it is not shipped with the package and rather saved on google drive. Here, we will use the 'opta' version.

In [1]:
library(data.table)
library("mlr3") # mlr3 base package
library("mlr3misc") # contains some helper functions
library("mlr3pipelines") # create ML pipelines
library("mlr3tuning") # tuning ML algorithms
library("mlr3learners") # additional ML algorithms
library("mlr3viz") # autoplot for benchmarks
library("paradox") # hyperparameter space
library("smotefamily") # SM

data_path = "/home/tarak/Gdrive_RA/events_data/opta"
training_data = readRDS(file.path(data_path, "training_dt.RDS"))

In [2]:
training_data =  training_data[, lapply(.SD, as.numeric)]
names(training_data)

[1] "type_id_a0"               "type_id_a1"              
  [3] "type_id_a2"               "type_pass_a0"            
  [5] "type_cross_a0"            "type_throw_in_a0"        
  [7] "type_freekick_crossed_a0" "type_freekick_short_a0"  
  [9] "type_corner_crossed_a0"   "type_corner_short_a0"    
 [11] "type_take_on_a0"          "type_foul_a0"            
 [13] "type_tackle_a0"           "type_interception_a0"    
 [15] "type_shot_a0"             "type_shot_penalty_a0"    
 [17] "type_shot_freekick_a0"    "type_keeper_save_a0"     
 [19] "type_keeper_claim_a0"     "type_keeper_punch_a0"    
 [21] "type_keeper_pick_up_a0"   "type_clearance_a0"       
 [23] "type_bad_touch_a0"        "type_non_action_a0"      
 [25] "type_dribble_a0"          "type_goalkick_a0"        
 [27] "type_pass_a1"             "type_cross_a1"           
 [29] "type_throw_in_a1"         "type_freekick_crossed_a1"
 [31] "type_freekick_short_a1"   "type_corner_crossed_a1"  
 [33] "type_corner_short_a1"     "type_take_on_a1"         
 [35] "type_foul_a1"             "type_tackle_a1"          
 [37] "type_interception_a1"     "type_shot_a1"            
 [39] "type_shot_penalty_a1"     "type_shot_freekick_a1"   
 [41] "type_keeper_save_a1"      "type_keeper_claim_a1"    
 [43] "type_keeper_punch_a1"     "type_keeper_pick_up_a1"  
 [45] "type_clearance_a1"        "type_bad_touch_a1"       
 [47] "type_non_action_a1"       "type_dribble_a1"         
 [49] "type_goalkick_a1"         "type_pass_a2"            
 [51] "type_cross_a2"            "type_throw_in_a2"        
 [53] "type_freekick_crossed_a2" "type_freekick_short_a2"  
 [55] "type_corner_crossed_a2"   "type_corner_short_a2"    
 [57] "type_take_on_a2"          "type_foul_a2"            
 [59] "type_tackle_a2"           "type_interception_a2"    
 [61] "type_shot_a2"             "type_shot_penalty_a2"    
 [63] "type_shot_freekick_a2"    "type_keeper_save_a2"     
 [65] "type_keeper_claim_a2"     "type_keeper_punch_a2"    
 [67] "type_keeper_pick_up_a2"   "type_clearance_a2"       
 [69] "type_bad_touch_a2"        "type_non_action_a2"      
 [71] "type_dribble_a2"          "type_goalkick_a2"        
 [73] "bodypart_foot_a0"         "bodypart_head_a0"        
 [75] "bodypart_other_a0"        "bodypart_foot_a1"        
 [77] "bodypart_head_a1"         "bodypart_other_a1"       
 [79] "bodypart_foot_a2"         "bodypart_head_a2"        
 [81] "bodypart_other_a2"        "result_id_a0"            
 [83] "result_id_a1"             "result_id_a2"            
 [85] "result_fail_a0"           "result_success_a0"       
 [87] "result_offside_a0"        "result_owngoal_a0"       
 [89] "result_yellow_card_a0"    "result_red_card_a0"      
 [91] "result_fail_a1"           "result_success_a1"       
 [93] "result_offside_a1"        "result_owngoal_a1"       
 [95] "result_yellow_card_a1"    "result_red_card_a1"      
 [97] "result_fail_a2"           "result_success_a2"       
 [99] "result_offside_a2"        "result_owngoal_a2"       
[101] "result_yellow_card_a2"    "result_red_card_a2"      
[103] "goalscore_team"           "goalscore_opponent"      
[105] "goalscore_diff"           "start_x_a0"              
[107] "start_y_a0"               "start_x_a1"              
[109] "start_y_a1"               "start_x_a2"              
[111] "start_y_a2"               "end_x_a0"                
[113] "end_y_a0"                 "end_x_a1"                
[115] "end_y_a1"                 "end_x_a2"                
[117] "end_y_a2"                 "dx_a0"                   
[119] "dy_a0"                    "movement_a0"             
[121] "dx_a1"                    "dy_a1"                   
[123] "movement_a1"              "dx_a2"                   
[125] "dy_a2"                    "movement_a2"             
[127] "dx_a01"                   "dy_a01"                  
[129] "mov_a01"                  "dx_a02"                  
[131] "dy_a02"                   "mov_a02"                 
[133] "start_dist_to_goal_a0"    "start_an

In [3]:
head(training_data, 3)

type_id_a0,type_id_a1,type_id_a2,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,⋯,time_seconds_overall_a1,period_id_a2,time_seconds_a2,time_seconds_overall_a2,time_delta_1,time_delta_2,scores,concedes,goal_from_shot,event_id
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,1,0,0,0,0,0,0,⋯,-2700,1,1,1,1,0,0,0,0,1.936e+09
0,0,0,1,0,0,0,0,0,0,⋯,1,0,0,-2700,1,2,0,0,0,1.235e+08
21,0,0,0,0,0,0,0,0,0,⋯,2,1,1,1,1,2,0,0,0,1.319e+09


The target columns should be transformed to meet `mlr3` requirement. We will also remove redundant columns (`type_id_` type) and create 2 binary classification tasks: one for the `scores` and another for the `concedes` target.

In [4]:

# columns to exclude anyway
to_exclude_anyway <- c("type_id_a0", "type_id_a1", "type_id_a2", "goal_from_shot") 

# score task
to_exclude_score <- c(to_exclude_anyway, "concedes")
dt_score <- training_data[, !..to_exclude_score]
dt_score[["scores"]] <- factor(ifelse(dt_score[["scores"]], "goal", "no_goal"), 
                               c("goal", "no_goal"))
task_score <- TaskClassif$new(id = "scores", backend = dt_score, target = "scores")
task_score$col_roles$name <- "event_id"
task_score$col_roles$feature <- setdiff(task_score$col_roles$feature, "event_id")

# concede task
to_exclude_concede <- c(to_exclude_anyway, "scores")
dt_concede = training_data[, !..to_exclude_concede]
dt_concede[["concedes"]] <- factor(ifelse(dt_concede[["concedes"]], "goal", "no_goal"), 
                                   c("goal", "no_goal"))
task_concede <- TaskClassif$new(id = "concedes", backend = dt_concede, target = "concedes")
task_concede$col_roles$name <- "event_id"
task_concede$col_roles$feature <- setdiff(task_concede$col_roles$feature, "event_id")

# Sampling Strategy

The data available is severly umbalanced as shown below:

In [5]:
table(task_score$truth())


   goal no_goal 
  18909 1395564 

In `mlr3pipelines`, there is a `classbalancing` and a `smote` pipe operator that can be combined with any learner. Below, we define the `undersampling`, `oversampling` and `SMOTE` `PipeOps`. All three imbalance correction methods have hyperparameters to control the degree of class imbalance. We apply the `PipeOps` to the current task with specific hyperparameter values to see how the class balance changes:

In [6]:
# undersample majority class (relative to majority class)
po_under = po("classbalancing", id = "undersample", 
              adjust = "major", 
              reference = "major", 
              shuffle = FALSE, ratio = 1 / 6)
# reduce majority class by factor '1/ratio'
table(po_under$train(list(task_score))$output$truth())


   goal no_goal 
  18909  232594 

In [7]:
# oversample majority class (relative to majority class)
po_over = po("classbalancing", id = "oversample", 
             adjust = "minor",
             reference = "minor", 
             shuffle = FALSE, 
             ratio = 6)
# enrich minority class by factor 'ratio'
table(po_over$train(list(task_score))$output$truth())


   goal no_goal 
 113454 1395564 

In [ ]:
# SMOTE enriches the minority class with synthetic data
po_smote = po("smote", dup_size = 2)
# enrich minority class by factor (dup_size + 1)
table(po_smote$train(list(task_score))$output$truth())